In [1]:
from infer import Infer
from utils.data_prepare import pipeline
import time

In [2]:
data = pipeline() #初始数据读取
trt_file_path = 'resnet.trt'
output_shape = [(1,1000)]
model = Infer(trt_file_path, output_shape)#初始化模型

Reading engine from file resnet.trt


In [3]:
%%time
for i in range(100):
    img = data.preprocess('test.jpg')
    _ = model(img)

CPU times: user 8.8 s, sys: 408 ms, total: 9.21 s
Wall time: 10.5 s


### 对比实验(1) 小模型效果

In [12]:
!time python3 trt_demo.py

Reading engine from file resnet.trt
加载模型用时：6.775
模型推理用时：10.718

real	0m20.127s
user	0m17.840s
sys	0m2.532s


In [13]:
!time python3 torch_demo.py

加载模型用时：5.065
模型推理用时：13.755


### 全流程示例

In [14]:
!cat utils/torch2onnx.py #定义网络模型并导出为onnx

import torch
import torchvision

model = torchvision.models.resnet152(pretrained=True)

batch_size = 1  #批处理大小
input_shape = (3, 244, 224)   #输入数据,改成自己的输入shape

# #set the model to inference mode
model.eval()

x = torch.randn(batch_size, *input_shape)	# 生成张量
export_onnx_file = "resnet152.onnx"			# 目的ONNX文件名
torch.onnx.export(model,
                    x,
                    export_onnx_file,
                    input_names=["input"],
                    output_names=["output"])

In [15]:
!python3 utils/torch2onnx.py #定义网络模型并导出为onnx

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /home/nvidia/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth
100.0%


In [19]:
!python3 onnx2trt.py resnet152.onnx resnet152.trt#将onnx模型转换为trt engine

Loading ONNX file from path resnet152.onnx...
Beginning ONNX file parsing
Completed parsing of ONNX file
Building an engine from file resnet152.onnx; this may take a while...
Completed creating Engine


### 对比实验（2） 大模型测试

In [20]:
### 修改对应的demo文件 改为resnet152

In [21]:
!time python3 trt_demo.py

Reading engine from file resnet152.trt
加载模型用时：8.510
模型推理用时：10.744

real	0m22.085s
user	0m18.740s
sys	0m3.272s


In [22]:
!time python3 torch_demo.py

加载模型用时：7.561
模型推理用时：20.700

real	0m30.569s
user	0m29.808s
sys	0m2.608s
